# Escolha do dataset

Para esta tarefa, vou usar o "[Spotify Tracks Dataset](https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset?resource=download)", que contém informações sobre músicas disponíveis no Spotify.



# Exploração do Dataset

## Características principais do dataset:


*   Número de amostras: Estou utilizando o número de amostras de 160.000.
*   Número de features: 19 características, danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo, etc.
*   Objetivo de classificação:  O objetivo é prever se uma música é "popular" (1) ou "não popular" (0), com base em um limiar de popularidade de 50, em uma escala de 0 a 100.





# Desenvolvimento do modelo

## Importando bibliotecas

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from google.colab import drive

## Conectando com drive para carregar dataset

In [7]:
drive.mount('/content/drive')
filepath = '/content/drive/MyDrive/ponderada_spotify/dataset.csv/dataset.csv'
df = pd.read_csv(filepath)

print(df.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   Unnamed: 0                track_id                 artists  \
0           0  5SuOikwiRyPMVoIQDJUgSV             Gen Hoshino   
1           1  4qPNDBW1i3p13qLCt0Ki3A            Ben Woodward   
2           2  1iJBSr7s7jYXzM8EGcbK5b  Ingrid Michaelson;ZAYN   
3           3  6lfxq3CG4xtTiEg7opyCyx            Kina Grannis   
4           4  5vjLSffimiIP26QG5WcN2K        Chord Overstreet   

                                          album_name  \
0                                             Comedy   
1                                   Ghost (Acoustic)   
2                                     To Begin Again   
3  Crazy Rich Asians (Original Motion Picture Sou...   
4                                            Hold On   

                   track_name  popularity  duration_ms  explicit  \
0                      Comedy          73       230666     False   
1      

## Seleção das features

In [8]:
features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
            'instrumentalness', 'liveness', 'valence', 'tempo']
X = df[features]


## Convertendo a popularidade em uma tarefa binária

In [9]:
y = df['popularity']
threshold = 50  # limiar de popularidade
y = np.where(y > threshold, 1, 0)

## Padronizando os dados

In [10]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## Dividindo em conjunto de treino e teste

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

## Definindo o modelo sequencial

In [12]:
model = Sequential()
model.add(Dense(1, activation='sigmoid', input_shape=(X_train.shape[1],)))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Compilando o modelo

In [14]:
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

## Treinando o modelo

In [15]:
history = model.fit(X_train, y_train, epochs=50, batch_size=10, validation_split=0.2, verbose=1)

Epoch 1/50
7296/7296 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.6848 - auc: 0.5564 - loss: 0.6138 - val_accuracy: 0.7562 - val_auc: 0.6201 - val_loss: 0.5393
Epoch 2/50
7296/7296 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.7566 - auc: 0.6091 - loss: 0.5414 - val_accuracy: 0.7561 - val_auc: 0.6194 - val_loss: 0.5386
Epoch 3/50
7296/7296 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.7574 - auc: 0.6164 - loss: 0.5385 - val_accuracy: 0.7561 - val_auc: 0.6179 - val_loss: 0.5388
Epoch 4/50
7296/7296 ━━━━━━━━━━━━━━━━━━━━ 29s 3ms/step - accuracy: 0.7573 - auc: 0.6160 - loss: 0.5386 - val_accuracy: 0.7561 - val_auc: 0.6186 - val_loss: 0.5388
Epoch 5/50
7296/7296 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.7589 - auc: 0.6170 - loss: 0.5364 - val_accuracy: 0.7561 - val_auc: 0.6167 - val_loss: 0.5393
Epoch 6/50
7296/7296 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - accuracy: 0.7577 - auc: 0.6121 - loss: 0.5390 - val_accuracy: 0.7561 - val_auc: 0.6188 - val_loss: 0.5395
Epoch 7/50
7296/7296 ━

## Avaliação no conjunto de teste

In [16]:
y_pred = model.predict(X_test)
y_pred_binary = np.round(y_pred)

713/713 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


## Calculando métricas

In [17]:
accuracy = accuracy_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')

Accuracy: 0.7564035087719299
F1 Score: 0.0


# Explicação


**Modelo Sequencial e Camada Dense**:é adequado para construir uma rede neural simples onde as camadas são empilhadas uma após a outra. Neste caso, usei uma única camada com um único neurônio para produzir uma saída binária (0 ou 1).

<br>


**Função de Ativação Sigmoid**: é usada para transformar a saída do neurônio em 0 e 1, ideal para tarefas de classificação binária.

<br>

**Otimizador Adam**: é um otimizador que ajusta a taxa de aprendizado durante o treinamento.

<br>

**Função de Perda Binary Crossentropy**: tem a função de medir a diferença entre as previsões do modelo e os valores reais. Logo, é ideal para tarefas de classificação binária.

<br>

**Métricas Accuracy e AUC**: mede a proporção de previsões corretas sobre o total de previsões. A AUC (Area Under the Curve) é usada para medir a capacidade do modelo de distinguir entre classes, neste caso, popular e não popular).

<br>

**Acurácia**: Isso indicará a proporção de músicas corretamente classificadas como populares ou não populares.

<br>

**F1 Score**: Fornece uma métrica mais equilibrada que leva em conta tanto a precisão quanto o recall, sendo especialmente útil em casos de dados desbalanceados.


# Resultados e Interpretação


**Desempenho**: uma acurácia de aproximadamente 75,6%,pode parecer bom. No entanto, o F1 score de 0,0 indica que o modelo não conseguiu identificar corretamente nenhuma das instâncias da classe positiva (músicas populares). Isso geralmente ocorre em cenários de dados desbalanceados, onde o modelo pode estar simplesmente prevendo todas as instâncias como a classe majoritária (neste caso, provavelmente a classe "não popular").

 <br>

**Possíveis Melhorias**:

1. Verificar Desbalanceamento: Conferir a distribuição das classes no seu dataset. Se o número de músicas populares for significativamente menor, isso explica o F1 score baixo.

2. Aplicar Técnicas de Balanceamento: Implementar técnicas de oversampling ou undersampling.

3. Reajustar o Modelo: Após balancear as classes, treine novamente o modelo e avalie o desempenho.